# <center> Sprint 10 </center>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time
import warnings
warnings.filterwarnings("ignore")

###  <center> Ejercicio 1 </center>

**<center> Web scraping de "quotes.toscrape" utilitzando BeautifulSoup. </center>**

- Guardo la url y compruebo su status

In [2]:
url='http://quotes.toscrape.com/'
resp=requests.get(url)

La solicitud es existosa, el status code está entre 200 y 209. 

- Obtengo el documento html con beautifulsoup usando html.parser para pasar la página web.

In [3]:
doc = BeautifulSoup(resp.text,'html.parser')
type(doc)

bs4.BeautifulSoup

Revisando la el contenedor con la información de las citas, encontramos una etiqueta "div" para cada cita, con la "class" "quote". Busco entonces todas las etiquetas "div" que tienen "class" "quote".  

In [4]:
div_etiqu =doc.find_all('div',class_='quote')
len(div_etiqu)

10

Cada página tiene 10 "quotes". div_etiqu contiene información como citas, nombre del autor, enlaces a la bibliografía del autor o etiquetas. 
- Voy a extraer las citas de la primera página. 

In [5]:
def get_citas(div_etiqu):
    citas=[]
    for tag in div_etiqu:
        cita=tag.find('span',class_='text').text
        citas.append(cita)
    return citas  

In [6]:
get_citas(div_etiqu)

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

- Obtengo el nombre del autor de las 10 citas. 

Se encuentran en la etiqueta "small" con "class=author" que está dentro de la etiqueta "span" con "class" None.

In [7]:
def get_nombre(div_etiqu):
    autores=[]
    for tag in div_etiqu:
        span_tag=tag.find('span',class_=None)
        autor=span_tag.find('small',class_='author').text
        autores.append(autor)
    return autores 

- Con el mismo procedimiento obtengo las etiquetas de las citas y los links a la biografia de los autores de una página.
    - Las etiquetas de las citas están en una etiqueta "meta" con atributo "content" el cual está dentro de la etiqueta "div" con "class=tag". 
    - Los links de la biografía del autor están dentro de una etiqueta con atributo "href" que a su vez está dentro de una etiqueta "span" con "class=None". 

In [8]:
def get_etiqcita(div_etiqu):
    nombre_etiquetas=[]
    for tag in div_etiqu:
        nombre_etiqueta=tag.find('div',class_='tags').meta['content']
        nombre_etiquetas.append(nombre_etiqueta)
    return nombre_etiquetas

In [9]:
def get_url_autor(div_etiqu):
    autor_links=[]
    for tag in div_etiqu:
        span_tag=tag.find('span',class_=None)
        autor_link='http://quotes.toscrape.com'+span_tag.find('a')['href']
        autor_links.append(autor_link)
    return autor_links

- Hago una lista de diccionarios combinando las listas obtenidas. 

In [10]:
def list_of_dict(citas_lista,nombre_autor,urls,nombre_etiqueta):
    return [{'Citas': citas_lista[i],
             'Autor': nombre_autor[i],
             'Etiquetas': nombre_etiqueta[i],
             'Link': urls[i]} for i in range(len(citas_lista))]
citas_dict=list_of_dict(get_citas(div_etiqu),get_nombre(div_etiqu),get_url_autor(div_etiqu),get_etiqcita(div_etiqu))

- Convierto el diccionario en un dataframe de pandas

In [11]:
Citas = pd.DataFrame(citas_dict)
Citas

,Citas,Autor,Etiquetas,Link
0,“The world as we have created it is a process ...,Albert Einstein,"change,deep-thoughts,thinking,world",http://quotes.toscrape.com/author/Albert-Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities,choices",http://quotes.toscrape.com/author/J-K-Rowling
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational,life,live,miracle,miracles",http://quotes.toscrape.com/author/Albert-Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy,books,classic,humor",http://quotes.toscrape.com/author/Jane-Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself,inspirational",http://quotes.toscrape.com/author/Marilyn-Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein,"adulthood,success,value",http://quotes.toscrape.com/author/Albert-Einstein
6,“It is better to be hated for what you are tha...,André Gide,"life,love",http://quotes.toscrape.com/author/Andre-Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"edison,failure,inspirational,paraphrased",http://quotes.toscrape.com/author/Thomas-A-Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,misattributed-eleanor-roosevelt,http://quotes.toscrape.com/author/Eleanor-Roos...
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"humor,obvious,simile",http://quotes.toscrape.com/author/Steve-Martin


**<center> Web scraping de "quotes.toscrape" utilitzando Selenium.**</center>

- Almaceno el sitio web en una variable

In [12]:
url = ('http://quotes.toscrape.com')

- Inicio el Chromedriver, abro el sitio web y maximizo la ventana

In [13]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

- Creo mis localizadores y los pruebo

In [14]:
Citas = driver.find_elements(by=By.XPATH, value="//div[@class='quote']/span[@class='text']")
Autores = driver.find_elements(by=By.XPATH, value="//div[@class='quote']/span[2]/small")
Tags = driver.find_elements(by=By.XPATH, value="//div[@class='quote']/div[@class='tags']")

In [15]:
for c in Citas:
    print(c.text)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


- Construyo el dataframe

In [16]:
dfs = []

for i in range(len(Citas)):
    data = {'Citas': Citas[i].text, 'Autores': Autores[i].text, 'Etiquetas': Tags[i].text}
    df_row = pd.DataFrame(data, index=[0]) 
    dfs.append(df_row)

df = pd.concat(dfs, ignore_index=True)

df

,Citas,Autores,Etiquetas
0,“The world as we have created it is a process ...,Albert Einstein,Tags: change deep-thoughts thinking world
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,Tags: abilities choices
2,“There are only two ways to live your life. On...,Albert Einstein,Tags: inspirational life live miracle miracles
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,Tags: aliteracy books classic humor
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,Tags: be-yourself inspirational
5,“Try not to become a man of success. Rather be...,Albert Einstein,Tags: adulthood success value
6,“It is better to be hated for what you are tha...,André Gide,Tags: life love
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,Tags: edison failure inspirational paraphrased
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,Tags: misattributed-eleanor-roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin,Tags: humor obvious simile


- Cierro el navegador para terminar la sesión 

In [17]:
driver.close()

**<center>Extraigo la lista de los 100 edificios más altos del mundo de la Wikipedia utilizando Selenium</center>**

- Almaceno el sitio web en la variable url2

In [18]:
url2 = ('https://es.wikipedia.org/wiki/Anexo:Rascacielos_m%C3%A1s_altos_del_mundo')

- Inicio el Chromedriver, abro el sitio web y maximizo la ventana

In [19]:
driverr = webdriver.Chrome()
driverr.get(url2)
driverr.maximize_window()

- Creo mis localizadores y verifico su medida

In [20]:
Nombre = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[3]")
Ciudad = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[4]")
Pais = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[5]")
Altura = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[6]")
Pisos = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[9]")
Año = driverr.find_elements(by=By.XPATH, value="//table[@class='wikitable sortable jquery-tablesorter'][1]/tbody/tr/td[10]")

In [21]:
len(Pisos)

100

- Construyo el dataframe final

In [22]:
dfs2 = []

for i in range(len(Nombre)):
    data = {'Nombre':Nombre[i].text, 'Ciudad':Ciudad[i].text, 'Pais':Pais[i].text, 
                                       'Altura':Altura[i].text, 'Pisos':Pisos[i].text, 'Año':Año[i].text}
    df_row = pd.DataFrame(data, index=[0])  
    dfs2.append(df_row)

df_Edificios = pd.concat(dfs2, ignore_index=True)

df_Edificios

,Nombre,Ciudad,Pais,Altura,Pisos,Año
0,Burj Khalifa,Dubái,Emiratos Árabes Unidos,828 m,163,2010
1,Merdeka 118,Kuala Lumpur,Malasia,679 m,118,2023
2,Torre de Shanghái,Shanghái,China,632 m,128,2015
3,Makkah Royal Clock Tower,La Meca,Arabia Saudita,601 m,120,2012
4,Ping An Finance Center,Shenzhen,China,599 m,115,2017
...,...,...,...,...,...,...
95,875 North Michigan Avenue,Chicago,Estados Unidos,344 m,100,1969
96,Shimao Global Financial Center,Changsha,China,343 m,71,2019
97,Four Seasons Place Kuala Lumpur,Kuala Lumpur,Malasia,343 m,65,2018
98,ADNOC Headquarters,Abu Dhabi,Emiratos Árabes Unidos,342 m,65,2015


- Cierro el navegador para terminar la sesión 

In [23]:
driverr.close()

**<center>Extraigo la lista de los 100 edificios más altos del mundo de la Wikipedia utilizando beautifulsoup</center>**

- Guardo la url y uso read_html para pasar la página web. Uso flavor con el motor ‘bs4’.

In [24]:
Edificios_df = pd.read_html(url2, flavor='bs4')

- Compruebo cantidad de tablas en la página web

In [25]:
len(Edificios_df)

5

- Extraigo la tabla que me interesa

In [26]:
Edificios_df[0]

Foto    #                         Edificio        Ciudad  \
   Foto    #                         Edificio        Ciudad   
0   NaN    1                     Burj Khalifa         Dubái   
1   NaN    2                      Merdeka 118  Kuala Lumpur   
2   NaN    3                Torre de Shanghái      Shanghái   
3   NaN    4         Makkah Royal Clock Tower       La Meca   
4   NaN    5           Ping An Finance Center      Shenzhen   
..  ...  ...                              ...           ...   
95  NaN   96        875 North Michigan Avenue       Chicago   
96  NaN   97   Shimao Global Financial Center      Changsha   
97  NaN   98  Four Seasons Place Kuala Lumpur  Kuala Lumpur   
98  NaN   99               ADNOC Headquarters     Abu Dhabi   
99  NaN  100         One Shenzhen Bay Tower 7      Shenzhen   

                      País  Altura                    Pisos   Año    Ref.  
                      País Oficial Antena Último piso Pisos   Año    Ref.  
0   Emiratos Árabes Unidos   828 m  830 m       585 m   163  2010    [3]​  
1                  Malasia   679 m  679 m       503 m   118  2023    [4]​  
2                    China   632 m  632 m       583 m   128  2015    [5]​  
3           Arabia Saudita   601 m  601 m       494 m   120  2012    [6]​  
4                    China   599 m  599 m       562 m   115  2017    [7]​  
..                     ...     ...    ...         ...   ...   ...     ...  
95          Estados Unidos   344 m  457 m       321 m   100  1969   [98]​  
96                   China   343 m  343 m       343 m    71  2019   [99]​  
97                 Malasia   343 m  343 m       343 m    65  2018  [100]​  
98  Emiratos Árabes Unidos   342 m  342 m       342 m    65  2015  [101]​  
99                   China   341 m  341 m       341 m    71  2018  [102]​  

[100 rows x 11 columns]

In [27]:
Edificios = pd.DataFrame(Edificios_df[0])

- Borro columnas innecesarias y obtengo el dataframe final.

In [28]:
Edificios.drop(['Foto','#', 'Ref.'], axis=1, inplace=True)

In [29]:
Edificios.columns

MultiIndex([('Edificio',    'Edificio'),
            (  'Ciudad',      'Ciudad'),
            (    'País',        'País'),
            (  'Altura',     'Oficial'),
            (  'Altura',      'Antena'),
            (  'Altura', 'Último piso'),
            (   'Pisos',       'Pisos'),
            (     'Año',         'Año')],
           )

In [30]:
Edificios.columns = ['Edificio', 'Ciudad', 'País', 'Altura oficial', 'Altura antena', 'Último piso', 'Pisos', 'Año']
Edificios

,Edificio,Ciudad,País,Altura oficial,Altura antena,Último piso,Pisos,Año
0,Burj Khalifa,Dubái,Emiratos Árabes Unidos,828 m,830 m,585 m,163,2010
1,Merdeka 118,Kuala Lumpur,Malasia,679 m,679 m,503 m,118,2023
2,Torre de Shanghái,Shanghái,China,632 m,632 m,583 m,128,2015
3,Makkah Royal Clock Tower,La Meca,Arabia Saudita,601 m,601 m,494 m,120,2012
4,Ping An Finance Center,Shenzhen,China,599 m,599 m,562 m,115,2017
...,...,...,...,...,...,...,...,...
95,875 North Michigan Avenue,Chicago,Estados Unidos,344 m,457 m,321 m,100,1969
96,Shimao Global Financial Center,Changsha,China,343 m,343 m,343 m,71,2019
97,Four Seasons Place Kuala Lumpur,Kuala Lumpur,Malasia,343 m,343 m,343 m,65,2018
98,ADNOC Headquarters,Abu Dhabi,Emiratos Árabes Unidos,342 m,342 m,342 m,65,2015


###  <center> Ejercicio 2 </center>

- En el pdf adjunto en la carpeta de Github

###  <center> Ejercicio 3 </center>

**<center>Extraigo la lista de los 10 quesos más consumidos del mundo de Escuela Gremio utilizando Selenium</center>**

- Almaceno el sitio web en la variable url3

In [31]:
url3 = ('https://escuelagremio.com/tipos-de-quesos-conocidos/')

- Inicio el Chromedriver, abro el sitio web y maximizo la ventana

In [32]:
drive = webdriver.Chrome()
drive.get(url3)
drive.maximize_window()

- Creo mis localizadores y los pruebo

In [33]:
Queso = drive.find_elements(by=By.XPATH, value="//h3[@style='text-align: justify;']")
Texto = drive.find_elements(by=By.XPATH, value="//p[@style='text-align: justify;']")
Descripción = Texto[5:15]

In [34]:
for q in Queso:
    print(q.text)

Mozzarella
Cheddar
Emmental
Gouda
Parmesano
Queso azul
Brie
Roquefort
Feta
Mascarpone


- Construyo el dataframe final

In [35]:
dfs3 = []

for i in range(len(Queso)):
    data = {'Tipo de queso':Queso[i].text,'Descripción':Descripción[i].text}
    df_row = pd.DataFrame(data, index=[0])  
    dfs3.append(df_row)

df_Quesos = pd.concat(dfs3, ignore_index=True)

df_Quesos

,Tipo de queso,Descripción
0,Mozzarella,Original de Italia y producida con leche de bú...
1,Cheddar,El Cheddar es original de Inglaterra y se elab...
2,Emmental,Este queso suizo se elabora con leche de vaca ...
3,Gouda,Este queso lleva el nombre de la ciudad holand...
4,Parmesano,"Comúnmente consumido en todo el mundo, el Parm..."
5,Queso azul,El queso azul puede elaborarse con leche de va...
6,Brie,Este conocido queso es original de la región d...
7,Roquefort,El Roquefort es un queso clásico francés. Se e...
8,Feta,El Feta es un queso griego que se elabora con ...
9,Mascarpone,El Mascarpone es un queso italiano distinto a ...


- Cierro el navegador para terminar la sesión 

In [36]:
drive.close()

**<center>Extraigo la lista de las citas de "quotes.toscrape" utilizando Scrapy</center>**

In [37]:
import scrapy, time
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import logging
import warnings
warnings.filterwarnings("ignore")

- Defino la clase "JsonWriterPipeline" para procesar los elementos del spider y convertir los elemntos a formato JSON con process:item. 

In [38]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('resultados.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

- Defino el spider CitasSpider y lo llamo "citas". Especifico URL (utilizo las dos primeras páginas) y saco el arcuivo en formato JSON (resultadocitas). En el método parse, defino la extracción de datos. Utilizo localizadores CSS para obtener los elementos específicos de la página: cita, autor y etiquetas.  

In [39]:
class CitasSpider(scrapy.Spider):
    name = "citas"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'resultadocitas.json'                        
    }
    
    def parse(self, response):
        for cita in response.css('div.quote'):
            yield {
                'text': cita.css('span.text::text').extract_first(),
                'author': cita.css('span small::text').extract_first(),
                'tags': cita.css('div.tags a.tag::text').extract(),
            }

**El archivo "resultadocitas.json" lo adjunto en la carpeta de Github.**

- Configuro y ejecuto el proceso de rastreo con CrawlerProcess. 

In [40]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(CitasSpider)
process.start()

2024-01-24 13:10:28 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-01-24 13:10:28 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 1.1.1w  11 Sep 2023), cryptography 41.0.3, Platform Windows-10-10.0.22631-SP0


- Cargo el archivo 'resultadocitas.json' en la variable data e imprimo cada resultado en la consola.

In [41]:
file_path = 'resultadocitas.json'
with open (file_path, 'r', encoding='utf-8') as file: 
    data = json.load(file)

for resultadocitas in data:
    print (resultadocitas)

{'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}
{'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': ['abilities', 'choices']}
{'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'author': 'Albert Einstein', 'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']}
{'text': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'author': 'Jane Austen', 'tags': ['aliteracy', 'books', 'classic', 'humor']}
{'text': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", 'author': 'Marilyn Monroe', 'tags': ['be-yourself', 'inspirational']}
